## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luganville,VU,-15.5333,167.1667,77.16,88,99,4.27,overcast clouds
1,1,Ellsworth,US,44.5434,-68.4195,54.12,25,0,10.36,clear sky
2,2,Rabo De Peixe,PT,37.8000,-25.5833,56.71,77,75,16.11,light rain
3,3,Vaini,TO,-21.2000,-175.2000,66.36,100,75,0.00,broken clouds
4,4,Talnakh,RU,69.4865,88.3972,-3.28,100,99,4.70,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luganville,VU,-15.5333,167.1667,77.16,88,99,4.27,overcast clouds
3,3,Vaini,TO,-21.2000,-175.2000,66.36,100,75,0.00,broken clouds
5,5,Kapaa,US,22.0752,-159.3190,76.48,84,20,13.80,few clouds
6,6,Rikitea,PF,-23.1203,-134.9692,75.18,74,100,4.99,light rain
8,8,Auka,HN,14.9333,-83.8333,86.14,61,77,6.44,broken clouds


In [16]:
# 4a. Determine if there are any empty rows.
#dropped na's in the step above
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#dropped na's in step # 3

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luganville,VU,77.16,overcast clouds,-15.5333,167.1667,
3,Vaini,TO,66.36,broken clouds,-21.2000,-175.2000,
5,Kapaa,US,76.48,few clouds,22.0752,-159.3190,
6,Rikitea,PF,75.18,light rain,-23.1203,-134.9692,
8,Auka,HN,86.14,broken clouds,14.9333,-83.8333,
10,Ishigaki,JP,77.13,scattered clouds,24.3448,124.1572,
13,Faanui,PF,80.44,overcast clouds,-16.4833,-151.7500,
15,Amapa,BR,73.99,overcast clouds,1.0000,-52.0000,
16,Rawson,AR,63.52,scattered clouds,-43.3002,-65.1023,
20,Butaritari,KI,81.64,broken clouds,3.0707,172.7902,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotels_df = hotel_df.replace(r'^\s*$',np.nan, regex=True)
clean_hotels_df
clean_hotel_df = clean_hotel_df.dropna().copy()
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Luganville,VU,77.16,overcast clouds,-15.5333,167.1667,The Espiritu Hotel
3,Vaini,TO,66.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Kapaa,US,76.48,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Rikitea,PF,75.18,light rain,-23.1203,-134.9692,People ThankYou
10,Ishigaki,JP,77.13,scattered clouds,24.3448,124.1572,Hotel Peace Land Ishigakijima
...,...,...,...,...,...,...,...
678,Banda Aceh,ID,81.39,broken clouds,5.5577,95.3222,OYO 884 Rumoh PMI Hotel
679,Iquique,CL,69.84,scattered clouds,-20.2208,-70.1431,Backpacker`s Hostel Iquique
680,Yerkoy,TR,65.50,scattered clouds,39.6381,34.4672,Yerköy Öğretmenevi
681,Barkhan,PK,75.99,scattered clouds,29.8977,69.5256,Sardar


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
